<a href="https://colab.research.google.com/github/ArunK-ML/Project---NASA-Near-Earth-Object-NEO-Tracking-Insights/blob/main/Nasa_Neo_streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

#Step 1: Getting the NASA API Key


import requests

asteroids_data = []
target = 10000
count = 0

url = "https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-07&api_key=E8laxB4HNUrbNSrlHmhopiwKuThHRsRXp1vQXGz9"

#Step 2 : Extract Data Using NASA's Asteroid API

while len(asteroids_data) < target:
    try:
        response = requests.get(url)
        nasa_data = response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error When Fetching Data: {e}")
        break

    details = nasa_data.get('near_earth_objects', {})
    if not details:
        print("No Data found in the Response. Exiting.....")
        break

    for date, asteroids in details.items():

        for ast in asteroids:

            # Try to extract close_approach_data safely

            approach_data = ast.get('close_approach_data', [])
            if approach_data:
                cad = approach_data[0]
                close_approach_date = cad.get('close_approach_date')
                try:
                    relative_velocity_kmph = float(cad['relative_velocity']['kilometers_per_hour'])
                except (KeyError, TypeError, ValueError):
                    relative_velocity_kmph = None

                try:
                    astronomical = float(cad['miss_distance']['astronomical'])
                    miss_distance_km = float(cad['miss_distance']['kilometers'])
                    miss_distance_lunar = float(cad['miss_distance']['lunar'])
                except (KeyError, TypeError, ValueError):
                    astronomical = None
                    miss_distance_km = None
                    miss_distance_lunar = None

                orbiting_body = cad.get('orbiting_body')

            else:
                close_approach_date = None
                relative_velocity_kmph = None
                astronomical = None
                miss_distance_km = None
                miss_distance_lunar = None
                orbiting_body = None

           # Step 3 :🧹 Data Cleaning Steps

            # Store all cleaned data into a dictionary

            asteroids_data.append({
                "sr_no": count,
                "id": ast.get('id'),
                "neo_id": ast.get("neo_reference_id"),
                "name": ast.get("name"),
                "absolute_magnitude_h": ast.get("absolute_magnitude_h"),
                "estimated_dia_min_km": ast['estimated_diameter']['kilometers'].get('estimated_diameter_min'),
                "estimated_dia_max_km": ast['estimated_diameter']['kilometers'].get('estimated_diameter_max'),
                "is_potentially_hazardous": ast.get("is_potentially_hazardous_asteroid"),
                "close_approach_date": close_approach_date,
                "relative_velocity_kmph": relative_velocity_kmph,
                "astronomical": astronomical,
                "miss_distance_km": miss_distance_km,
                "miss_distance_lunar": miss_distance_lunar,
                "orbiting_body": orbiting_body
            })

            count += 1
            if len(asteroids_data) == target:
                break
        if len(asteroids_data) == target:
            break

    # Get next page URL from NASA API

    url = nasa_data['links'].get('next')
    if not url:
        break

# Output: Summary

print(f"\n✅ Collected {len(asteroids_data)} asteroids (target was {target}).")

# Output: First 10 asteroids

print("\n📌 First 10 Asteroids:")
for i, ast in enumerate(asteroids_data[:10]):
    print(f"\n--- Asteroid {i+1} ---")
    for key, value in ast.items():
        print(f"{key}: {value}")


✅ Collected 10000 asteroids (target was 10000).

📌 First 10 Asteroids:

--- Asteroid 1 ---
sr_no: 0
id: 2415949
neo_id: 2415949
name: 415949 (2001 XY10)
absolute_magnitude_h: 19.37
estimated_dia_min_km: 0.3552670883
estimated_dia_max_km: 0.7944013596
is_potentially_hazardous: False
close_approach_date: 2024-01-02
relative_velocity_kmph: 57205.8951204341
astronomical: 0.3372535274
miss_distance_km: 50452409.349026635
miss_distance_lunar: 131.1916221586
orbiting_body: Earth

--- Asteroid 2 ---
sr_no: 1
id: 3160747
neo_id: 3160747
name: (2003 SR84)
absolute_magnitude_h: 26.0
estimated_dia_min_km: 0.0167708462
estimated_dia_max_km: 0.0375007522
is_potentially_hazardous: False
close_approach_date: 2024-01-02
relative_velocity_kmph: 38589.054833182
astronomical: 0.1323425924
miss_distance_km: 19798169.933318187
miss_distance_lunar: 51.4812684436
orbiting_body: Earth

--- Asteroid 3 ---
sr_no: 2
id: 3309828
neo_id: 3309828
name: (2005 YQ96)
absolute_magnitude_h: 20.62
estimated_dia_min_km: 0

# **Step 4: Insert NASA Asteroid Data into SQL**

In [2]:
import sqlite3

conn = sqlite3.connect("Asteroids_Data_File.db")
cursor = conn.cursor()

In [3]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS asteroids (
    id INTEGER,
    name TEXT,
    absolute_magnitude_h REAL,
    estimated_dia_min_km REAL,
    estimated_dia_max_km REAL,
    is_potentially_hazardous BOOLEAN
    )
""")
conn.commit()

In [4]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS close_approach (
    neo_reference_id INTEGER,
    close_approach_date DATE,
    relative_velocity_kmph REAL,
    astronomical REAL,
    miss_distance_km REAL,
    miss_distance_lunar REAL,
    orbiting_body TEXT
)
""")
conn.commit()

# **Prepare insert Values in tables**

In [5]:
# Prepare insert query

insert_query = ("""
INSERT INTO asteroids (
    id,
    name,
    absolute_magnitude_h,
    estimated_dia_min_km,
    estimated_dia_max_km,
    is_potentially_hazardous
) VALUES (?, ?, ?, ?, ?, ?)
""")

# Insert each asteroid
for ast in asteroids_data:
    cursor.execute(insert_query, (
        int(ast['id']) if ast['id'] else None,
        ast['name'],
        ast['absolute_magnitude_h'],
        ast['estimated_dia_min_km'],
        ast['estimated_dia_max_km'],
        ast['is_potentially_hazardous']
    ))

# Commit
conn.commit()


print("✅ Asteroid data inserted successfully!")

✅ Asteroid data inserted successfully!


In [6]:
insert_query = ("""
INSERT INTO close_approach (
    neo_reference_id,
    close_approach_date,
    relative_velocity_kmph,
    astronomical,
    miss_distance_km,
    miss_distance_lunar,
    orbiting_body
) VALUES (?, ?, ?, ?, ?, ?, ?)
""")

# Insert each asteroid
for ast in asteroids_data:
    cursor.execute(insert_query, (
        ast["neo_id"],
        ast["close_approach_date"],
        ast["relative_velocity_kmph"],
        ast["astronomical"],
        ast["miss_distance_km"],
        ast["miss_distance_lunar"],
        ast["orbiting_body"]
    ))

# Commit
conn.commit()


print("✅ Close_approach data inserted successfully!")

✅ Close_approach data inserted successfully!


# **SQL Tables info**

In [7]:
cursor.execute("SELECT COUNT(*) FROM asteroids;")
count1 = cursor.fetchone()[0]
print(f"✅ Total records in table: {count1}")

✅ Total records in table: 10000


In [8]:
cursor.execute("SELECT COUNT(*) FROM close_approach;")
count2 = cursor.fetchone()[0]
print(f"✅ Total records in table: {count2}")

✅ Total records in table: 10000


In [9]:
cursor.execute("SELECT * FROM asteroids")

data1 = cursor.fetchall()

#fetches the names of your columns in dataframe(cursor.description holds the columns info)

columns = [i[0] for i in cursor.description]

import pandas as pd

# using the result fetched in cursor, create a new dataframe

asteroids_df = pd.DataFrame(data1,columns=columns)

asteroids_df # shows all the values inserted - 7 columns

,id,name,absolute_magnitude_h,estimated_dia_min_km,estimated_dia_max_km,is_potentially_hazardous
0,2415949,415949 (2001 XY10),19.37,0.355267,0.794401,0
1,3160747,(2003 SR84),26.00,0.016771,0.037501,0
2,3309828,(2005 YQ96),20.62,0.199781,0.446725,1
3,3457842,(2009 HC21),22.10,0.101054,0.225964,0
4,3553062,(2010 XA11),26.10,0.016016,0.035813,0
...,...,...,...,...,...,...
9995,54016746,(2020 HB6),24.20,0.038420,0.085909,0
9996,54134936,(2021 GU1),24.71,0.030378,0.067927,0
9997,54340154,(2023 BM4),23.71,0.048145,0.107656,0
9998,54380881,(2023 RX1),30.25,0.002369,0.005297,0


In [10]:
cursor.execute("SELECT * FROM close_approach")

data2 = cursor.fetchall()

#fetches the names of your columns in dataframe(cursor.description holds the columns info)

columns = [i[0] for i in cursor.description]

import pandas as pd

# using the result fetched in cursor, create a new dataframe

close_approach_df = pd.DataFrame(data2,columns=columns)

close_approach_df # shows all the values inserted - 7 columns

,neo_reference_id,close_approach_date,relative_velocity_kmph,astronomical,miss_distance_km,miss_distance_lunar,orbiting_body
0,2415949,2024-01-02,57205.895120,0.337254,5.045241e+07,131.191622,Earth
1,3160747,2024-01-02,38589.054833,0.132343,1.979817e+07,51.481268,Earth
2,3309828,2024-01-02,56413.014352,0.167013,2.498473e+07,64.967910,Earth
3,3457842,2024-01-02,21891.118219,0.492051,7.360980e+07,191.407879,Earth
4,3553062,2024-01-02,31468.978359,0.235802,3.527551e+07,91.727075,Earth
...,...,...,...,...,...,...,...
9995,54016746,2025-04-13,74844.228017,0.415699,6.218768e+07,161.706907,Earth
9996,54134936,2025-04-13,78326.810991,0.416653,6.233035e+07,162.077878,Earth
9997,54340154,2025-04-13,18299.183030,0.164612,2.462559e+07,64.034019,Earth
9998,54380881,2025-04-13,4963.996594,0.046395,6.940661e+06,18.047832,Earth


In [11]:
asteroids_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        10000 non-null  int64  
 1   name                      10000 non-null  object 
 2   absolute_magnitude_h      10000 non-null  float64
 3   estimated_dia_min_km      10000 non-null  float64
 4   estimated_dia_max_km      10000 non-null  float64
 5   is_potentially_hazardous  10000 non-null  int64  
dtypes: float64(3), int64(2), object(1)
memory usage: 468.9+ KB


In [12]:
close_approach_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   neo_reference_id        10000 non-null  int64  
 1   close_approach_date     10000 non-null  object 
 2   relative_velocity_kmph  10000 non-null  float64
 3   astronomical            10000 non-null  float64
 4   miss_distance_km        10000 non-null  float64
 5   miss_distance_lunar     10000 non-null  float64
 6   orbiting_body           10000 non-null  object 
dtypes: float64(4), int64(1), object(2)
memory usage: 547.0+ KB


In [13]:
# To corret the date data type

close_approach_df['close_approach_date'] = pd.to_datetime(close_approach_df['close_approach_date'])

close_approach_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   neo_reference_id        10000 non-null  int64         
 1   close_approach_date     10000 non-null  datetime64[ns]
 2   relative_velocity_kmph  10000 non-null  float64       
 3   astronomical            10000 non-null  float64       
 4   miss_distance_km        10000 non-null  float64       
 5   miss_distance_lunar     10000 non-null  float64       
 6   orbiting_body           10000 non-null  object        
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 547.0+ KB


# **Step 5 : SQL Queries (Display the output in streamlit)**

In [ ]:
#1. Count how many times each asteroid has approached Earth
#2. Average velocity of each asteroid over multiple approaches
#3. List top 10 fastest asteroids
#4. Find potentially hazardous asteroids that have approached Earth more than 3 times
#5. Find the month with the most asteroid approaches
#6. Get the asteroid with the fastest ever approach speed
#7. Sort asteroids by maximum estimated diameter (descending)
#8. An asteroid whose closest approach is getting nearer over time(Hint: Use ORDER BY close_approach_date and look at miss_distance).
#9. Display the name of each asteroid along with the date and miss distance of its closest approach to Earth.
#10. List names of asteroids that approached Earth with velocity > 50,000 km/h
#11. Count how many approaches happened per month
#12. Find asteroid with the highest brightness (lowest magnitude value)
#13. Get number of hazardous vs non-hazardous asteroids
#14. Find asteroids that passed closer than the Moon (lesser than 1 LD), along with their close approach date and distance.
#15. Find asteroids that came within 0.05 AU(astronomical distance)

In [14]:
import sqlite3
import pandas as pd

# Connect to your database
connection = sqlite3.connect('Asteroids_Data_File.db')
cursor = connection.cursor()

def show_query(query):
    df = pd.read_sql_query(query, connection)
    display(df)

In [15]:
#1. Count how many times each asteroid has approached Earth

query1 = ("""
SELECT neo_reference_id, COUNT(*) AS approach_count
FROM close_approach
GROUP BY neo_reference_id
ORDER BY approach_count DESC
""")
show_query(query1)

,neo_reference_id,approach_count
0,54356994,5
1,54161317,5
2,3989284,5
3,3840791,5
4,3771633,5
...,...,...
7372,2006239,1
7373,2005660,1
7374,2005645,1
7375,2001685,1


In [ ]:
#2. Average velocity of each asteroid over multiple approaches

query2 = ("""
SELECT neo_reference_id, AVG(relative_velocity_kmph) AS avg_velocity
FROM close_approach
GROUP BY neo_reference_id
ORDER BY avg_velocity DESC
""")
show_query(query2)

,neo_reference_id,avg_velocity
0,54392072,173071.830081
1,3803884,161520.897925
2,3790060,158279.864701
3,3425295,154336.407297
4,3092220,152218.220518
...,...,...
7375,54485079,3040.309519
7376,54459240,2149.995258
7377,3798020,1909.578168
7378,54440316,1610.566846


In [ ]:
#3. List top 10 fastest asteroids

query3 = ("""
SELECT neo_reference_id, AVG(relative_velocity_kmph) AS avg_velocity
FROM close_approach
GROUP BY neo_reference_id
ORDER BY avg_velocity DESC
LIMIT 10
""")
show_query(query3)

,neo_reference_id,avg_velocity
0,54392072,173071.830081
1,3803884,161520.897925
2,3790060,158279.864701
3,3425295,154336.407297
4,3092220,152218.220518
5,3588900,151520.395439
6,54448599,150599.454097
7,54111253,150583.019419
8,3383901,149261.278634
9,2412977,148185.748026


In [ ]:
#4. Find potentially hazardous asteroids that have approached Earth more than 3 times

query4 = ("""
SELECT a.name, COUNT(ca.neo_reference_id) AS approach_count
FROM close_approach AS ca
JOIN asteroids AS a ON ca.neo_reference_id = a.id
WHERE a.is_potentially_hazardous = 1
GROUP BY a.name
HAVING COUNT(ca.neo_reference_id) > 3
""")
show_query(query4)

,name,approach_count
0,(2003 GQ22),4
1,(2004 TP1),9
2,(2005 TU50),4
3,(2006 QQ23),4
4,(2006 TU7),9
...,...,...
90,613986 (2008 JG),9
91,65679 (1989 UQ),4
92,675603 (2015 XE352),4
93,741232 (2005 TU50),4


In [ ]:
#5. Find the month with the most asteroid approaches

query5 = ("""
SELECT strftime('%m', close_approach_date) AS month, COUNT(*) AS approach_count
FROM close_approach
GROUP BY month
ORDER BY approach_count DESC
LIMIT 1
""")
show_query(query5)

,month,approach_count
0,03,1290


In [ ]:
#6. Get the asteroid with the fastest ever approach speed

query6 = ("""
SELECT a.name, ca.relative_velocity_kmph
FROM asteroids a
JOIN close_approach ca ON a.id = ca.neo_reference_id
ORDER BY ca.relative_velocity_kmph DESC
LIMIT 1
""")
show_query(query6)

,name,relative_velocity_kmph
0,(2023 TS3),173071.830081


In [ ]:
#7. Sort asteroids by maximum estimated diameter (descending)

query7 = ("""
SELECT name, estimated_dia_max_km
FROM asteroids
ORDER BY estimated_dia_max_km DESC
""")
show_query(query7)

,name,estimated_dia_max_km
0,887 Alinda (A918 AA),10.233872
1,1685 Toro (1948 OA),8.280184
2,66146 (1998 TU3),7.980681
3,5660 (1974 MA),4.989304
4,533671 (2014 LJ21),3.926811
...,...,...
9995,(2011 CQ1),0.002260
9996,(2024 YW8),0.002208
9997,(2023 RS),0.002042
9998,(2024 RC42),0.001986


In [ ]:
#8. An asteroid whose closest approach is getting nearer over time(Hint: Use ORDER BY close_approach_date and look at miss_distance).

query8 = ("""
SELECT a.name, ca.close_approach_date, ca.miss_distance_km
FROM close_approach ca
JOIN asteroids a ON ca.neo_reference_id = a.id
ORDER BY ca.neo_reference_id, ca.close_approach_date
""")
show_query(query8)

,name,close_approach_date,miss_distance_km
0,887 Alinda (A918 AA),2025-01-08,1.229661e+07
1,1685 Toro (1948 OA),2024-01-20,1.990593e+07
2,2063 Bacchus (1977 HB),2024-03-31,1.795253e+07
3,2063 Bacchus (1977 HB),2024-03-31,1.795253e+07
4,2063 Bacchus (1977 HB),2024-03-31,1.795253e+07
...,...,...,...
16315,(2025 MG89),2024-07-21,1.167117e+07
16316,(2025 MB90),2024-07-17,2.015873e+07
16317,(2025 MB90),2024-07-17,2.015873e+07
16318,(2025 MB90),2024-07-17,2.015873e+07


In [ ]:
#9. Display the name of each asteroid along with the date and miss distance of its closest approach to Earth.

query9 = ("""
SELECT a.name, ca.close_approach_date, ca.miss_distance_km
FROM close_approach ca
JOIN asteroids a ON ca.neo_reference_id = a.id
WHERE ca.miss_distance_km = (
    SELECT MIN(ca2.miss_distance_km)
    FROM close_approach ca2
    WHERE ca2.neo_reference_id = ca.neo_reference_id
)
""")
show_query(query9)

,name,close_approach_date,miss_distance_km
0,415949 (2001 XY10),2024-01-02,5.045241e+07
1,415949 (2001 XY10),2024-01-02,5.045241e+07
2,(2003 SR84),2024-01-02,1.979817e+07
3,(2005 YQ96),2024-01-02,2.498473e+07
4,(2009 HC21),2024-01-02,7.360980e+07
...,...,...,...
16315,(2025 GX),2025-04-09,3.061345e+06
16316,(2025 GA1),2025-04-09,4.794783e+06
16317,(2025 HX1),2025-04-09,7.531629e+06
16318,(2015 AQ45),2025-04-13,2.267146e+07


In [ ]:
#10. List names of asteroids that approached Earth with velocity > 50,000 km/h

query10 = ("""
SELECT DISTINCT a.name
FROM close_approach ca
JOIN asteroids a ON ca.neo_reference_id = a.id
WHERE ca.relative_velocity_kmph > 50000
""")
show_query(query10)

,name
0,415949 (2001 XY10)
1,(2005 YQ96)
2,(2017 YD8)
3,(2019 KK5)
4,613286 (2005 YQ96)
...,...
2874,(2008 FW6)
2875,(2017 PK25)
2876,(2022 PX)
2877,(2022 SW)


In [ ]:
#11. Count how many approaches happened per month

query11 = ("""
SELECT strftime('%m', close_approach_date) AS month, COUNT(*) AS approach_count
FROM close_approach
GROUP BY month
ORDER BY month
""")
show_query(query11)

,month,approach_count
0,01,1238
1,02,1156
2,03,1290
3,04,948
4,05,577
5,06,527
6,07,471
7,08,621
8,09,853
9,10,903


In [ ]:
#12. Find asteroid with the highest brightness (lowest magnitude value)

query12 = ("""
SELECT name, absolute_magnitude_h
FROM asteroids
ORDER BY absolute_magnitude_h ASC
LIMIT 1
""")
show_query(query12)

,name,absolute_magnitude_h
0,887 Alinda (A918 AA),13.82


In [ ]:
#13. Get number of hazardous vs non-hazardous asteroids

query13 = ("""
SELECT
    CASE
        WHEN is_potentially_hazardous = 1 THEN 'Hazardous'
        ELSE 'Non-Hazardous'
    END AS asteroid_type,
    COUNT(*) AS count
FROM asteroids
GROUP BY asteroid_type
""")
show_query(query13)

,asteroid_type,count
0,Hazardous,498
1,Non-Hazardous,9502


In [ ]:
#14. Find asteroids that passed closer than the Moon (lesser than 1 LD), along with their close approach date and distance.

query14 = ("""
SELECT a.name, ca.close_approach_date, ca.miss_distance_lunar
FROM close_approach ca
JOIN asteroids a ON ca.neo_reference_id = a.id
WHERE ca.miss_distance_lunar < 1
""")
show_query(query14)

,name,close_approach_date,miss_distance_lunar
0,(2024 AD),2024-01-04,0.631031
1,(2024 AG4),2024-01-11,0.885273
2,(2024 AM4),2024-01-11,0.310675
3,(2024 BA16),2024-01-15,0.364068
4,(2024 AZ3),2024-01-14,0.647173
...,...,...,...
300,(2025 FV12),2025-03-28,0.264304
301,(2025 FM18),2025-04-02,0.721706
302,(2025 GA),2025-04-02,0.221098
303,(2025 GS),2025-04-05,0.444223


In [ ]:
#15. Find asteroids that came within 0.05 AU(astronomical distance)

query15 = ("""
SELECT a.name, ca.close_approach_date, ca.astronomical
FROM close_approach ca
JOIN asteroids a ON ca.neo_reference_id = a.id
WHERE ca.astronomical < 0.05
""")
show_query(query15)

,name,close_approach_date,astronomical
0,(2019 KK5),2024-01-02,0.026097
1,(2023 YR),2024-01-02,0.011564
2,(2024 AA),2024-01-02,0.004081
3,(2024 AQ1),2024-01-02,0.043829
4,(2023 YR1),2024-01-01,0.016608
...,...,...,...
3609,(2025 GR1),2025-04-08,0.011719
3610,(2025 HA),2025-04-08,0.049300
3611,(2025 HJ3),2025-04-08,0.032458
3612,(2025 GX),2025-04-09,0.020464


In [ ]:
# Extra Queries

In [ ]:
#16.Find asteroids with an average speed > 40,000 km/h

query16 = ("""
SELECT a.name, AVG(ca.relative_velocity_kmph) AS avg_velocity
FROM asteroids a
JOIN close_approach ca ON a.id = ca.neo_reference_id
GROUP BY a.id
HAVING avg_velocity > 40000;
""")
show_query(query16)


,name,avg_velocity
0,1685 Toro (1948 OA),58226.721077
1,2340 Hathor (1976 UA),93669.384688
2,4034 Vishnu (1986 PA),43128.810179
3,5645 (1990 SP),66107.059559
4,5660 (1974 MA),77045.190447
...,...,...
3845,(2025 HQ4),53799.364737
3846,(2025 HP6),87566.492183
3847,(2025 HD43),40474.985693
3848,(2025 KR1),55701.962785


In [ ]:
#17.Calculate average miss distance for each asteroid

query17 = ("""
SELECT neo_reference_id, AVG(miss_distance_km) AS avg_miss_distance_km
FROM close_approach
GROUP BY neo_reference_id;
""")
show_query(query17)

,neo_reference_id,avg_miss_distance_km
0,2000887,1.229661e+07
1,2001685,1.990593e+07
2,2002063,1.795253e+07
3,2002340,5.948137e+07
4,2004034,2.345060e+07
...,...,...
7375,54533502,6.763441e+06
7376,54533907,1.386360e+07
7377,54536909,9.703563e+06
7378,54537502,1.167117e+07


In [ ]:
#18.List top 5 largest potentially hazardous asteroids (by estimated max diameter)

query18 = ("""
SELECT name, estimated_dia_min_km
FROM asteroids
WHERE is_potentially_hazardous = 1
ORDER BY estimated_dia_min_km DESC
LIMIT 5;
""")
show_query(query18)

,name,estimated_dia_min_km
0,533671 (2014 LJ21),1.756123
1,415029 (2011 UL21),1.732028
2,163243 (2002 FB3),1.350688
3,458122 (2010 EW45),0.821401
4,504034 (2005 UJ159),0.817627


In [ ]:
#19.Find asteroids that approached Earth exactly once

query19 = ("""
SELECT a.name
FROM asteroids a
JOIN close_approach ca ON a.id = ca.neo_reference_id
GROUP BY a.id
HAVING COUNT(*) = 1;
""")
show_query(query19)

,name
0,887 Alinda (A918 AA)
1,1685 Toro (1948 OA)
2,5645 (1990 SP)
3,5660 (1974 MA)
4,6239 Minos (1989 QF)
...,...
5218,(2025 KW2)
5219,(2025 JB3)
5220,(2025 KZ8)
5221,(2025 MU34)


In [ ]:
#20. Average size (diameter) of hazardous vs. non-hazardous asteroids

query20 = ("""
SELECT is_potentially_hazardous,AVG(estimated_dia_min_km + estimated_dia_max_km) / 2 AS avg_diameter_km
FROM asteroids
GROUP BY is_potentially_hazardous;
""")
show_query(query20)

,is_potentially_hazardous,avg_diameter_km
0,0,0.094429
1,1,0.375039


In [ ]:
#21. Find asteroids that approached Earth more than once in the same day

query21 = ("""
SELECT neo_reference_id, close_approach_date, COUNT(*) AS approaches
FROM close_approach
GROUP BY neo_reference_id, close_approach_date
HAVING approaches > 1;
""")
show_query(query21)

,neo_reference_id,close_approach_date,approaches
0,2002063,2024-03-31,2
1,2002340,2024-11-02,2
2,2004034,2025-04-01,2
3,2065679,2024-10-15,2
4,2137126,2025-03-08,2
...,...,...,...
1444,54528330,2024-10-27,2
1445,54529811,2025-01-13,2
1446,54533318,2024-12-08,2
1447,54533502,2024-05-12,2


In [ ]:
#22. Find all asteroids that have never been hazardous

query22 = ("""
SELECT id, name
FROM asteroids
WHERE is_potentially_hazardous = 0;
""")
show_query(query22)

,id,name
0,2415949,415949 (2001 XY10)
1,3160747,(2003 SR84)
2,3457842,(2009 HC21)
3,3553062,(2010 XA11)
4,3591616,(2011 YP10)
...,...,...
9497,54525110,(2025 FW12)
9498,54526424,(2025 GX)
9499,54526427,(2025 GA1)
9500,54527383,(2025 HX1)


In [ ]:
#23. Which day had the most asteroid approaches?

query23 = ("""
SELECT close_approach_date, COUNT(*) AS total_approaches
FROM close_approach
GROUP BY close_approach_date
ORDER BY total_approaches DESC
LIMIT 1;
""")
show_query(query23)

,close_approach_date,total_approaches
0,2024-09-27,66


In [ ]:
#24. List asteroids that only approached Earth in a specific year (e.g : 2025)

query24 = ("""
SELECT DISTINCT a.name
FROM asteroids a
JOIN close_approach ca ON a.id = ca.neo_reference_id
WHERE strftime('%Y', ca.close_approach_date) = '2025';
""")
show_query(query24)

,name
0,(2011 GE3)
1,(2021 NT5)
2,(2021 VR3)
3,(2022 EQ6)
4,(2024 YV1)
...,...
1705,(2024 UZ9)
1706,(2025 FW12)
1707,(2025 GX)
1708,(2025 GA1)


In [ ]:
#25. Find the asteroid with the most total approaches

query25 = ("""
SELECT neo_reference_id, COUNT(*) AS total_approaches
FROM close_approach
GROUP BY neo_reference_id
ORDER BY total_approaches DESC
LIMIT 1;
""")
show_query(query25)


,neo_reference_id,total_approaches
0,54356994,5


# **Streamlit **

In [16]:
%%writefile streamlit_app.py
# streamlit_app.py
import streamlit as st
import sqlite3
import pandas as pd
import plotly.express as px

st.set_page_config(layout="wide")

# Connect to SQLite DB
def get_db_connection():
    return sqlite3.connect("Asteroids_Data_File.db")

# Execute SQL Query
def run_query(query):
    conn = get_db_connection()
    try:
        df = pd.read_sql(query, conn)
        return df
    except Exception as e:
        st.error(f"Error executing query: {e}")
        return None
    finally:
        conn.close()

# Sidebar Navigation
st.sidebar.title("🌠 Asteroid Dashboard Menu")
page = st.sidebar.radio("Navigate", ["Home", "Filter Criteria", "Queries", "About"])

# Query Lists
guvi_questions = [
    "1. Count how many times each asteroid has approached Earth",
    "2. Average velocity of each asteroid over multiple approaches",
    "3. List top 10 fastest asteroids",
    "4. Find potentially hazardous asteroids that have approached Earth more than 3 times",
    "5. Find the month with the most asteroid approaches",
    "6. Get the asteroid with the fastest ever approach speed",
    "7. Sort asteroids by maximum estimated diameter (descending)",
    "8. An asteroid whose closest approach is getting nearer over time",
    "9. Display the name of each asteroid along with the date and miss distance of its closest approach to Earth.",
    "10. List names of asteroids that approached Earth with velocity > 50,000 km/h",
    "11. Count how many approaches happened per month",
    "12. Find asteroid with the highest brightness (lowest magnitude value)",
    "13. Get number of hazardous vs non-hazardous asteroids",
    "14. Find asteroids that passed closer than the Moon (less than 1 LD)",
    "15. Find asteroids that came within 0.05 AU"
]

guvi_queries = [
    "SELECT neo_reference_id, COUNT(*) AS approach_count FROM close_approach GROUP BY neo_reference_id ORDER BY approach_count DESC",
    "SELECT neo_reference_id, AVG(relative_velocity_kmph) AS avg_velocity FROM close_approach GROUP BY neo_reference_id ORDER BY avg_velocity DESC",
    "SELECT neo_reference_id, AVG(relative_velocity_kmph) AS avg_velocity FROM close_approach GROUP BY neo_reference_id ORDER BY avg_velocity DESC LIMIT 10",
    "SELECT a.name, COUNT(ca.neo_reference_id) AS approach_count FROM close_approach AS ca JOIN asteroids AS a ON ca.neo_reference_id = a.id WHERE a.is_potentially_hazardous = 1 GROUP BY a.name HAVING COUNT(ca.neo_reference_id) > 3",
    "SELECT strftime('%m', close_approach_date) AS month, COUNT(*) AS approach_count FROM close_approach GROUP BY month ORDER BY approach_count DESC LIMIT 1",
    "SELECT a.name, ca.relative_velocity_kmph FROM asteroids a JOIN close_approach ca ON a.id = ca.neo_reference_id ORDER BY ca.relative_velocity_kmph DESC LIMIT 1",
    "SELECT name, estimated_dia_max_km FROM asteroids ORDER BY estimated_dia_max_km DESC",
    "SELECT a.name, ca.close_approach_date, ca.miss_distance_km FROM close_approach ca JOIN asteroids a ON ca.neo_reference_id = a.id ORDER BY ca.neo_reference_id, ca.close_approach_date",
    "SELECT a.name, ca.close_approach_date, ca.miss_distance_km FROM close_approach ca JOIN asteroids a ON ca.neo_reference_id = a.id WHERE ca.miss_distance_km = (SELECT MIN(ca2.miss_distance_km) FROM close_approach ca2 WHERE ca2.neo_reference_id = ca.neo_reference_id)",
    "SELECT DISTINCT a.name FROM close_approach ca JOIN asteroids a ON ca.neo_reference_id = a.id WHERE ca.relative_velocity_kmph > 50000",
    "SELECT strftime('%m', close_approach_date) AS month, COUNT(*) AS approach_count FROM close_approach GROUP BY month ORDER BY month",
    "SELECT name, absolute_magnitude_h FROM asteroids ORDER BY absolute_magnitude_h ASC LIMIT 1",
    "SELECT CASE WHEN is_potentially_hazardous = 1 THEN 'Hazardous' ELSE 'Non-Hazardous' END AS asteroid_type, COUNT(*) AS count FROM asteroids GROUP BY asteroid_type",
    "SELECT a.name, ca.close_approach_date, ca.miss_distance_lunar FROM close_approach ca JOIN asteroids a ON ca.neo_reference_id = a.id WHERE ca.miss_distance_lunar < 1",
    "SELECT a.name, ca.close_approach_date, ca.astronomical FROM close_approach ca JOIN asteroids a ON ca.neo_reference_id = a.id WHERE ca.astronomical < 0.05"
]

my_queries = [
    "16. Find asteroids with an average speed > 40,000 km/h",
    "17. Calculate average miss distance for each asteroid",
    "18. List top 5 largest potentially hazardous asteroids",
    "19. Find asteroids that approached Earth exactly once",
    "20. Average size (diameter) of hazardous vs. non-hazardous asteroids",
    "21. Asteroids with multiple approaches on the same day",
    "22. All asteroids that have never been hazardous",
    "23. Day with most asteroid approaches",
    "24. Asteroids that only approached Earth in 2025",
    "25. Asteroid with most total approaches"
]

my_queries_list = [
    "SELECT a.name, AVG(ca.relative_velocity_kmph) AS avg_velocity FROM asteroids a JOIN close_approach ca ON a.id = ca.neo_reference_id GROUP BY a.id HAVING avg_velocity > 40000;",
    "SELECT neo_reference_id, AVG(miss_distance_km) AS avg_miss_distance_km FROM close_approach GROUP BY neo_reference_id;",
    "SELECT name, estimated_dia_min_km FROM asteroids WHERE is_potentially_hazardous = 1 ORDER BY estimated_dia_min_km DESC LIMIT 5;",
    "SELECT a.name FROM asteroids a JOIN close_approach ca ON a.id = ca.neo_reference_id GROUP BY a.id HAVING COUNT(*) = 1;",
    "SELECT is_potentially_hazardous, AVG((estimated_dia_min_km + estimated_dia_max_km) / 2) AS avg_diameter_km FROM asteroids GROUP BY is_potentially_hazardous;",
    "SELECT neo_reference_id, close_approach_date, COUNT(*) AS approaches FROM close_approach GROUP BY neo_reference_id, close_approach_date HAVING approaches > 1;",
    "SELECT id, name FROM asteroids WHERE is_potentially_hazardous = 0;",
    "SELECT close_approach_date, COUNT(*) AS total_approaches FROM close_approach GROUP BY close_approach_date ORDER BY total_approaches DESC LIMIT 1;",
    "SELECT DISTINCT a.name FROM asteroids a JOIN close_approach ca ON a.id = ca.neo_reference_id WHERE strftime('%Y', ca.close_approach_date) = '2025';",
    "SELECT neo_reference_id, COUNT(*) AS total_approaches FROM close_approach GROUP BY neo_reference_id ORDER BY total_approaches DESC LIMIT 1;"
]

# === Page Logic ===
st.title("☄️ Asteroid Insights Explorer")

if page == "Home":
    st.header("Welcome to the Asteroid Data Dashboard")

    col1, col2 = st.columns([2,2])  # Wider text column on the left

    with col1:
        st.markdown("""
        ### 🚀 Explore NASA's Near-Earth Object Data
        This dashboard allows you to:
        - View asteroids' close approaches
        - Explore their speeds, sizes, and distances
        - Check which are potentially hazardous
        - Download filtered results for further analysis

        Navigate using the sidebar to run queries or apply custom filters.
        """)

    with col2:
        st.image("/content/170275-843069693_tiny.jpg")

elif page == "Filter Criteria":
    st.header("Filter Criteria")
    st.info("Use the filter options below to explore specific asteroid characteristics.")

    col1, col2, col3 = st.columns(3)
    with col1:
        sl_min_magnitude = st.slider("Min Magnitude", 13.80, 32.61, (13.80, 20.34))
    with col2:
        sl_relative_velocity = st.slider("Relative Velocity (km/h)", 1418.21, 173071.83, (1600.00, 50000.00))
    with col3:
        start_date = st.date_input("Start Date")

    col4, col5, col6 = st.columns(3)
    with col4:
        sl_min_estimated_dia_range = st.slider("Min Estimated Diameter (km)", 0.00, 4.62, (1.0, 2.10))
    with col5:
        sl_max_estimated_dia_range = st.slider("Max Estimated Diameter (km)", 0.00, 10.33, (1.0, 6.0))
    with col6:
        end_date = st.date_input("End Date")

    col7, col8 = st.columns(2)
    with col7:
        sl_astronomical_unit = st.slider("Astronomical Unit", 0.00, 0.58, (0.05, 0.48))
    with col8:
        sl_potentially_hazardous = st.selectbox("Is Potentially Hazardous", [0, 1])

    if st.button("Filter"):
        query = f"""
        SELECT a.name, ca.close_approach_date, ca.relative_velocity_kmph, ca.astronomical,
               ca.miss_distance_km, a.absolute_magnitude_h, a.estimated_dia_max_km, a.is_potentially_hazardous
        FROM close_approach ca
        JOIN asteroids a ON a.id = ca.neo_reference_id
        WHERE a.absolute_magnitude_h BETWEEN {sl_min_magnitude[0]} AND {sl_min_magnitude[1]}
          AND a.estimated_dia_min_km BETWEEN {sl_min_estimated_dia_range[0]} AND {sl_min_estimated_dia_range[1]}
          AND a.estimated_dia_max_km BETWEEN {sl_max_estimated_dia_range[0]} AND {sl_max_estimated_dia_range[1]}
          AND ca.relative_velocity_kmph BETWEEN {sl_relative_velocity[0]} AND {sl_relative_velocity[1]}
          AND ca.astronomical BETWEEN {sl_astronomical_unit[0]} AND {sl_astronomical_unit[1]}
          AND a.is_potentially_hazardous = {sl_potentially_hazardous}
          AND date(ca.close_approach_date) BETWEEN '{start_date}' AND '{end_date}'
        """
        df = run_query(query)
        if df is not None and not df.empty:
            st.dataframe(df, use_container_width=True)
            fig = px.histogram(df, x="relative_velocity_kmph", title="Velocity Distribution")
            st.plotly_chart(fig, use_container_width=True)
            st.download_button("Download CSV", df.to_csv(index=False), "filtered_results.csv")
        else:
            st.warning("No matching data found for selected criteria.")

elif page == "Queries":
    st.header("Predefined SQL Queries")
    category = st.radio("Choose Query Type", ["Guvi Queries", "My Queries"])
    if category == "Guvi Queries":
        desc = st.selectbox("Select Query:", guvi_questions)
        query = guvi_queries[guvi_questions.index(desc)]
    else:
        desc = st.selectbox("Select Query:", my_queries)
        query = my_queries_list[my_queries.index(desc)]

    if st.button("Run Query"):
        df = run_query(query)
        if df is not None and not df.empty:
            st.subheader(desc)
            st.dataframe(df, use_container_width=True)
            numeric_cols = df.select_dtypes(include=["int64", "float64"]).columns
            if len(numeric_cols) > 0:
                col = st.selectbox("Select column to plot", numeric_cols)
                fig = px.bar(df, x=df.columns[0], y=col, title="Visual Representation")
                st.plotly_chart(fig, use_container_width=True)
        else:
            st.warning("No data found or query failed.")

elif page == "About":
    st.header("About This App")
    st.write("Built using Streamlit, this app allows exploration of near-Earth asteroid data with SQL and interactive charts.")
    st.write("Data Collected through Nasa API Link")
    st.write("**Project Given by Guvi**")
    st.write("**Thank you for visit**")


st.markdown("---")
st.caption("Developed by Arun Kumar | Powered by NASA NEO Data")


Writing streamlit_app.py


In [17]:
!pip install streamlit streamlit_option_menu  # installing streamlit and streamlit_option_menu packages

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 98.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.3/829.3 kB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 101.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.3 MB/s eta 0:00:00


In [18]:
!wget -q -O - ipv4.icanhazip.com   # this command will generate a password for you(copy that)

35.245.125.3


In [19]:
!streamlit run streamlit_app.py & npx localtunnel --port 8501  # replace the file name accordingly

⠙⠹⠸⠼

⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.245.125.3:8501

⠦⠧Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇your url is: https://wicked-rocks-wish.loca.lt
  Stopping...
^C
